# Imports

In [2]:
# add to sys.path
import sys

sys.path.insert(0, "../")
sys.path.insert(0, "../src")

In [3]:
import time
import logging
import pandas as pd
from datetime import datetime, timedelta, timezone
from dateutil.relativedelta import relativedelta
import plaid


# gcp
from google.cloud import bigquery
from google.cloud import pubsub_v1
from google.cloud import scheduler_v1
from google.cloud import functions_v2
from google.cloud.functions_v1.types import ListFunctionsRequest
from google.protobuf import duration_pb2

# SCHEMAS
from jobs.bq_table_schemas import BqTableSchemas
from jobs.cloud_schemas import CloudSchemas


# UTILS
from utils.bq_utils import BqUtils
from utils.gcp_utils import GcpUtils
from utils.plaid_utils import PlaidUtils
from utils.secrets_utils import SecretsUtils
from utils.crypto_utils import CryptoUtils
from utils.budget_values import BudgetValues
from utils.financial_accounts import FinancialAccounts
from utils.plaid_transactions import PlaidTransactions
from utils.plaid_investments import PlaidInvestments


# get all secrets
sec = SecretsUtils()
secrets = sec.create_secrets_dict(plaid_secret_type="PROD")
PLAID_CLIENT_ID = secrets["PLAID_CLIENT_ID"]
PLAID_SECRET = secrets["PLAID_SECRET_PROD"]
PLAID_ACCESS_TOKENS = sec.get_access_token_secrets(secrets)
PLAID_HOST = plaid.Environment.Production
PLAID_PRODUCTS = ["liabilities", "transactions", "investments"]
PLAID_COUNTRY_CODES = ["US"]
CRYPTO_SECRETS = secrets["CRYPTO_SECRETS"]

# initialize clients
bq_client = bigquery.Client()
bq = BqUtils(bq_client=bq_client)
plaid_client = PlaidUtils(bq_client, PLAID_CLIENT_ID, PLAID_SECRET, PLAID_HOST)
financial_accounts = FinancialAccounts(bq_client, plaid_client)
plaid_transactions = PlaidTransactions(bq_client, plaid_client)
plaid_investments = PlaidInvestments(bq_client, plaid_client)
budget_values = BudgetValues(bq_client)
crypto = CryptoUtils()
bq_tables = BqTableSchemas()
gcp = GcpUtils(bq_client=bq_client)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", 1000)

# Empty Transactions

In [3]:
# need to create public_budget_values() to get access to budget values
bv = budget_values.public_budget_values()
date = "2024-04-01"

def empty_transactions_sql(date):

    categories_added = set()

    query = ""

    for i in bv:
        if (i["category"] not in categories_added and i["category_raw"] != "EXCLUDE_CATEGORY"):
            query += f"""
{"UNION ALL" if len(categories_added) != 0 else ""}
SELECT 
    FORMAT_DATE("%Y%m%d", "{date}") AS partition_date,
    "EMPTY_TRANSACTION" AS item_id,
    "EMPTY_TRANSACTION" AS account_id,
    "EMPTY_TRANSACTION" AS transaction_id,
    CAST("{date}" AS DATE) AS transaction_date,
    CAST(NULL AS DATETIME) AS transaction_datetime,
    DATE_TRUNC(CAST("{date}" AS DATE), MONTH) AS transaction_month,
    "DEBIT" AS transaction_type,
    0.0 AS actual_amount,
    "USD" AS currency_code,
    "{i["category_raw"]}" AS category_raw,
    "{i["subcategory_raw"]}" AS subcategory_raw,
    "{i["category"]}" AS category_updated,
    "{i["subcategory"]}" AS subcategory_updated,
    "other" AS payment_channel,
    CAST(NULL AS STRING) AS merchant_name_raw,
    CAST(NULL AS STRING) AS counterparty_name_raw,
    CAST(NULL AS STRING) AS name_raw,
    CAST(NULL AS STRING) AS merchant_type,
            """

            categories_added.add(i["category"])

            # print(categories_added)
    
    return query

empty_transactions_query = empty_transactions_sql(date)
# print(empty_transactions_query)